In [1]:
import urllib.request 
from json import *
import pandas as pd
import numpy as np
import datetime as dt
import time
import matplotlib.dates as mdates

In [ ]:
#Data from Fantasy Premier League API

u = "http://fantasy.premierleague.com/web/api/elements/4"
x1 = urllib.request.urlopen(u).read().decode()
dic = {}
dic = JSONDecoder().decode(x1)
print(dic)

playerGameweek_df = pd.DataFrame(dic['fixture_history']['all'])
playerGameweek_df.columns = ['Date','Round','Opponent','Minutes_Played','Goals_Scored','Assists','Clean_Sheets',
                             'Goals_Conceded','Own_goals','Penalties_Saved','Penalties_Missed',
                             'Yellow_Cards','Red_Cards','Saves','Bonus','PPI','BonusPS','Net_Transfers',
                             'Player_Value','Points']
playerGameweek_df.set_index('Round', inplace=True, drop=True)

playerPreviousSeason_df = pd.DataFrame(dic['season_history'])
playerPreviousSeason_df.columns = ['Year','Minutes_Played','Goals_Scored','Assists','Clean_Sheets',
                             'Goals_Conceded','Own_goals','Penalties_Saved','Penalties_Missed',
                             'Yellow_Cards','Red_Cards','Saves','Bonus','PPI','BonusPS','Player_Value','Points']
#playerPreviousSeason_df.set_index('Year', inplace=True, drop=True)

playerGenInfor_df = pd.Series()
dicIndex = ['type_name','first_name','web_name','team_name', 'dreamteam_count'
            ,'chance_of_playing_this_round','selected_by_percent', 'next_fixture']
for index in dicIndex:
    joiner = pd.Series(dic[index], name = str(index))
    playerGenInfor_df = pd.concat([playerGenInfor_df,joiner], axis = 1)

print(playerGenInfor_df.head(5))
print(playerPreviousSeason_df.head(5))
print(playerGameweek_df.head(5))

In [2]:
#Data from Premier League Website

def league_table(season, month, tm, club = 'All'):
    tmUnix = time.mktime(dt.datetime.strptime(tm, '%d/%m/%y').timetuple())
    tmUnix = str(tmUnix*1000)
    url = 'http://www.premierleague.com/en-gb/matchday/league-table.html?season='+season+'&month='+month+'&timelineView=date&toDate='+tmUnix+'&tableView=HOME_VS_AWAY'
    df = pd.read_html(url)
    df = pd.DataFrame(df[0])
    table = pd.DataFrame(df)
    col_names = ['Pos', 'CLUB', 'HP', 'HW', 'HD', 'HL', 'HGF', 'HGA', 'HGD', 'HPTS', 'AP', 'AW', 'AD', 'AL', 'AGF', 'AGA', 'AGD', 'APTS', 'TGD', 'TPTS']
    table = table.transpose()
    table.dropna(inplace = True)
    table = table.transpose()
    table.columns = col_names
    table = table.iloc[0:20]
    timestamp = [[(np.int(np.float(tmUnix)))] * len(table)]
    timeDic = pd.Series(timestamp[0], name = 'timestamp')
    table = table.join(timeDic)
    table.set_index('Pos', inplace = True, drop=False)    
    if club != 'All':
        table = table.iloc[:][table.iloc[:,1] == club]
    return table

#league_table('YYYY-YYYY','Full Month in caps', '%d/%m/%y', Optional = 'Team name')                              
#leagueTable = league_table('2015-2016','JANUARY', 'Jan-03-2016 18:59:59')
#d = dt.datetime.fromtimestamp(int(1451865599)).strftime('%b-%d-%Y %H:%M:%S')
#print(time.mktime(dt.datetime.strptime(d, '%b-%d-%Y %H:%M:%S').timetuple()))

In [3]:
df = pd.read_csv('C:/Users/ArchishmanMitra/Documents/Data Science/Python Scripts/FPL Team of the Week Prediction/E0.csv')
date = df['Date'].iloc[50]
tm = time.mktime(dt.datetime.strptime(date, '%d/%m/%y').timetuple())
resultTable = df[['Date', 'HomeTeam', 'AwayTeam', 'FTR']]
#print(len(resultTable))
ComTable = pd.DataFrame()
AllFinalDf = pd.DataFrame()
for matches in range(len(resultTable)):
    if resultTable.Date[matches][3:5] == '08':
        month = 'AUGUST'
    if resultTable.Date[matches][3:5] == '09':
        month = 'SEPTEMBER'
    if resultTable.Date[matches][3:5] == '10':
        month = 'OCTOBER'
    if resultTable.Date[matches][3:5] == '11':
        month = 'NOVEMBER'
    if resultTable.Date[matches][3:5] == '12':
        month = 'DECEMBER'
    if resultTable.Date[matches][3:5] == '01':
        month = 'OCTOBER'
    leagueTable = league_table('2015-2016',month, resultTable.Date[matches], resultTable.HomeTeam[matches])
    leagueTableJoiner  = league_table('2015-2016',month, resultTable.Date[matches], resultTable.AwayTeam[matches])
    #FinalDf = pd.concat([leagueTable,leagueTableJoiner], ignore_index = True)
    ComTable = pd.merge(leagueTable,leagueTableJoiner, on = 'timestamp')
    ComTable['Result'] = resultTable.FTR[matches]
    AllFinalDf = pd.concat([AllFinalDf,ComTable], ignore_index = True)
AllFinalDf.to_pickle('Final Results.pickle')
AllFinalDf.describe()
#2 mins for 20
#6 mins for 50

,timestamp
count,1.900000e+02
mean,1.445508e+12
std,3.967348e+09
min,1.439006e+12
25%,1.442117e+12
50%,1.445659e+12
75%,1.449292e+12
max,1.451452e+12


In [5]:
df = pd.read_pickle('Final Results.pickle')
df

,Pos_x,CLUB_x,HP_x,HW_x,HD_x,HL_x,HGF_x,HGA_x,HGD_x,HPTS_x,...,AW_y,AD_y,AL_y,AGF_y,AGA_y,AGD_y,APTS_y,TGD_y,TPTS_y,Result
0,11,Bournemouth,2,0,1,1,1,2,-1,1,...,1,0,1,2,2,0,3,-1,4,A
1,13,Chelsea,2,0,1,1,3,4,-1,1,...,0,2,0,3,3,0,2,3,8,D
2,9,Everton,2,0,1,1,2,4,-2,1,...,0,1,1,2,4,-2,1,-2,3,D
3,3,Leicester City,2,1,1,0,5,3,2,4,...,0,1,1,4,6,-2,1,-4,2,H
4,5,Manchester United,2,1,1,0,1,0,1,4,...,0,1,1,1,2,-1,1,-1,3,H
5,14,Norwich City,2,0,1,1,2,4,-2,1,...,2,0,0,5,2,3,6,3,9,A
6,6,Arsenal,2,0,1,1,0,2,-2,1,...,2,0,0,5,0,5,6,3,6,A
7,19,Newcastle United,2,0,1,1,2,3,-1,1,...,0,2,0,2,2,0,2,0,5,D
8,18,Stoke City,2,0,0,2,0,2,-2,0,...,1,1,0,1,0,1,4,-1,7,A
9,15,West Bromwich Albion,2,0,0,2,2,6,-4,0,...,2,0,0,5,0,5,6,10,12,A
